In [14]:
!bq load --source_format NEWLINE_DELIMITED_JSON --autodetect \
--ignore_unknown_values sourabhjainceanalytics:demo.target_table gs://bigquery_json_processing/target_table.json

Waiting on bqjob_r51fa21b5c197bbec_0000017e4dfdc7a7_1 ... (1s) Current status: DONE   


In [15]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.demo.target_table`

Downloading: 100%|██████████| 5/5 [00:00<00:00,  5.79rows/s]


,region,name,salary,id
0,Mumbai,Sourabh,100,1
1,Mumbai,Rajesh,200,2
2,Mumbai,Nishit,300,3
3,Mumbai,Rakesh,400,4
4,Mumbai,Vivek,500,5


In [16]:
!bq load --source_format NEWLINE_DELIMITED_JSON --autodetect \
--ignore_unknown_values sourabhjainceanalytics:demo.change_table gs://bigquery_json_processing/change_table.json

Waiting on bqjob_r768e0d2f5c15b5b4_0000017e4dfdddd0_1 ... (0s) Current status: DONE   


In [17]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.demo.change_table`

Downloading: 100%|██████████| 3/3 [00:00<00:00,  3.60rows/s]


,region,salary,id
0,None,1000.0,1
1,Ahmedabad,NaN,2
2,Chennai,3000.0,3


In [18]:
%%bigquery
MERGE demo.target_table T
USING demo.change_table S
ON T.id = S.id
WHEN MATCHED THEN 
        UPDATE SET   T.region = CASE WHEN S.region IS NULL THEN T.region ELSE S.region END
                    ,T.salary = CASE WHEN S.salary IS NULL THEN T.salary ELSE S.salary END

Query complete after 0.00s: 100%|██████████| 6/6 [00:00<00:00, 3327.05query/s]                        


""


In [19]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.demo.target_table`

Downloading: 100%|██████████| 5/5 [00:00<00:00,  5.88rows/s]


,region,name,salary,id
0,Mumbai,Sourabh,1000,1
1,Mumbai,Rakesh,400,4
2,Mumbai,Vivek,500,5
3,Chennai,Nishit,3000,3
4,Ahmedabad,Rajesh,200,2


In [20]:
%%bigquery
SELECT column_name 
FROM sourabhjainceanalytics.demo.INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('change_table') 
and column_name NOT IN ('id')

Downloading: 100%|██████████| 2/2 [00:00<00:00,  2.44rows/s]


,column_name
0,region
1,salary


In [21]:
%%bigquery
SELECT * FROM `sourabhjainceanalytics.demo.target_table`

Downloading: 100%|██████████| 5/5 [00:00<00:00,  6.24rows/s]


,region,name,salary,id
0,Mumbai,Sourabh,1000,1
1,Mumbai,Rakesh,400,4
2,Mumbai,Vivek,500,5
3,Chennai,Nishit,3000,3
4,Ahmedabad,Rajesh,200,2
